In [1]:
import datasets

datasets.downloads = '/home/greg/Downloads/'

trX, teX, trY, teY = datasets.mnist()

In [2]:
import numpy as np

In [22]:
decode = lambda p: np.argmax(p, axis = 1) 
labels = decode(teY)
epochs = 1
H = 625
N, D = trX.shape
N, C = trY.shape
accuracy = lambda p: np.mean(decode(p) == labels)

In [46]:
import tensorflow as tf

def layer(x, shape, activator):
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return activator(tf.matmul(x, w) + b)  
    
def fit(sess, x, y, X, Y, optimizer, epochs):
    sess.run(tf.initialize_all_variables())

    for _ in range(epochs):
        for start in range(0, N - 32, 32):
            end = start + 32
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

In [47]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
h = layer(x, [D, H], tf.nn.relu)  
model = layer(h, [H, C], tf.nn.softmax)  
loss = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    fit(sess, x, y, trX, trY, optimizer, epochs)
    print accuracy(model.eval({x: teX}))

0.9375


In [49]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = H, input_dim = D, activation = 'relu'))
model.add(Dense(output_dim = C, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = epochs)
print accuracy(model.predict_proba(teX, verbose = 0))

Epoch 1/1
60000/60000 [==============================] - 23s - loss: 0.1942    
0.9642


In [186]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)